**Общая информация**

**Срок сдачи:** 13 марта 2017, 06:00 <br\>
**Штраф за опоздание:** -2 балла после 06:00 13 марта, -4 балла после 06:00 20 марта, -6 баллов после 06:00 27 марта

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack создать таск в приватный чат:
/todo Фамилия Имя ссылка на гитхаб @alkhamush
Пример:
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе.

###### Задание 2 (2 балла)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn 4 балла для iris и mnist
Для этого используем numpy

###### Задание 3 (2 балла)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 3 (2 балла)
Выполнить требования pep8

###### Задание 5 (2 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
#%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyKNeighborsClassifier(NeighborsBase,
                             KNeighborsMixin,
                             SupervisedIntegerMixin,
                             ClassifierMixin):

    def __init__(self, n_neighbors):
        self.neighbors_param = n_neighbors

    def fit(self, X, y):
        self.fit_X = X.copy()
        self.fit_y = y.copy()
        self.fit_classes = []
        for t in self.fit_y:
            if t not in self.fit_classes:
                self.fit_classes.append(t)

    def predict(self, X):
        y = np.zeros(X.shape[0])
        fit_X_tmp = np.tile(self.fit_X, (X.shape[0], 1))
        X_tmp = np.tile(X, (1, self.fit_X.shape[0]))
        X_tmp = X_tmp.reshape((self.fit_X.shape[0]*X.shape[0], X.shape[1]))
        tmp = (((fit_X_tmp - X_tmp) ** 2).sum(axis=1)) ** 0.5
        tmp = tmp.reshape((X.shape[0], self.fit_X.shape[0]))
        tmp = np.argsort(tmp, axis=1)
        tmp = tmp[:, 0:self.neighbors_param].copy()
        rows_mask = np.arange(tmp.shape[0]).reshape(tmp.shape[0], 1)
        rows_mask = np.tile(rows_mask, (1, self.neighbors_param))
        cols_mask = tmp
        print("cols_mask_dim = %s, rows_mask_dim = %s" % (cols_mask.shape, rows_mask.shape))
        fit_y_tmp = self.fit_y.reshape((1, self.fit_y.shape[0]))
        fit_y_tmp = np.tile(fit_y_tmp, (tmp.shape[0], 1))
        tttmp = np.zeros((tmp.shape[0], len(self.fit_classes)))
        for i, t in enumerate(self.fit_classes):
            ttttmp = np.zeros((fit_y_tmp.shape[0], fit_y_tmp.shape[1]))
            ttttmp[fit_y_tmp == t] = 1
            tttmp[:, i] = ttttmp[rows_mask, cols_mask].sum(axis=1)
        tttmp = np.argsort(tttmp, axis=1)
        y = tttmp[:, -1].copy()
        tmp_classes = np.array(self.fit_classes)
        y = tmp_classes[y].copy()
        return y

    def predict_proba(self, X):
        y = np.zeros(X.shape[0])
        fit_X_tmp = np.tile(self.fit_X, (X.shape[0], 1))
        X_tmp = np.tile(X, (1, self.fit_X.shape[0]))
        X_tmp = X_tmp.reshape((self.fit_X.shape[0]*X.shape[0], X.shape[1]))
        tmp = (((fit_X_tmp - X_tmp) ** 2).sum(axis=1)) ** 0.5
        tmp = tmp.reshape((X.shape[0], self.fit_X.shape[0]))
        tmp = np.argsort(tmp, axis=1)
        tmp = tmp[:, 0:self.neighbors_param].copy()
        rows_mask = np.arange(tmp.shape[0]).reshape(tmp.shape[0], 1)
        rows_mask = np.tile(rows_mask, (1, self.neighbors_param))
        cols_mask = tmp
        fit_y_tmp = self.fit_y.reshape((1, self.fit_y.shape[0]))
        fit_y_tmp = np.tile(fit_y_tmp, (tmp.shape[0], 1))
        tttmp = np.zeros((tmp.shape[0], len(self.fit_classes)))
        for i, t in enumerate(self.fit_classes):
            ttttmp = np.zeros((fit_y_tmp.shape[0], fit_y_tmp.shape[1]))
            ttttmp[fit_y_tmp == t] = 1
            tttmp[:, i] = ttttmp[rows_mask, cols_mask].sum(axis=1)
        tmp_sums = tttmp.sum(axis=1).reshape(tttmp.shape[0], 1)
        tmp_sums = np.tile(tmp_sums, (1, tttmp.shape[1]))
        tttmp = tttmp / tmp_sums

        rows_mask = np.arange(tttmp.shape[0]).reshape(tttmp.shape[0], 1)
        rows_mask = np.tile(rows_mask, (1, len(self.fit_classes)))
        cols_mask = np.arange(len(self.fit_classes))
        cols_mask = np.array(cols_mask)[np.argsort(np.array(self.fit_classes))]
        cols_mask = np.tile(cols_mask, (tttmp.shape[0], 1))
        y = tttmp[rows_mask, cols_mask].copy()
        return y

    def score(self, X, y):
        predicted = self.predict(X)
        cnt = 0.
        for i, t in enumerate(y):
            if predicted[i] == t:
                cnt = cnt + 1.
        return cnt/len(y)


**IRIS**

In [3]:
iris = datasets.load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [5]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [6]:
%time clf.fit(X_train, y_train)

CPU times: user 5.12 ms, sys: 333 µs, total: 5.45 ms
Wall time: 5.83 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [7]:
%time my_clf.fit(X_train, y_train)

CPU times: user 577 µs, sys: 0 ns, total: 577 µs
Wall time: 483 µs


In [8]:
%time clf.predict(X_test)

CPU times: user 956 µs, sys: 265 µs, total: 1.22 ms
Wall time: 995 µs


array([1, 0, 1, 0, 0, 1, 2, 1, 1, 2, 2, 1, 0, 0, 2])

In [9]:
%time my_clf.predict(X_test)

cols_mask_dim = (15, 2), rows_mask_dim = (15, 2)
CPU times: user 2.42 ms, sys: 0 ns, total: 2.42 ms
Wall time: 1.61 ms


array([1, 0, 1, 0, 0, 1, 2, 1, 1, 2, 2, 1, 0, 0, 2])

In [10]:
%time clf.predict_proba(X_test)

CPU times: user 1.14 ms, sys: 314 µs, total: 1.45 ms
Wall time: 1.03 ms


array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [11]:
%time my_clf.predict_proba(X_test)

CPU times: user 438 µs, sys: 123 µs, total: 561 µs
Wall time: 569 µs


array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [12]:
clf.score(X_test, y_test)

0.8

In [14]:
my_clf.score(X_test, y_test)

cols_mask_dim = (15, 2), rows_mask_dim = (15, 2)


0.8

In [14]:
# Задание 3
neighbors = range(1, 100)
accur = []
for i in neighbors:
    print(i)
    my_clf = MyKNeighborsClassifier(n_neighbors=i)
    my_clf.fit(X_train, y_train)
    accur.append(my_clf.score(X_test, y_test))

plt.plot(neighbors, accur, color="blue", label="score")
plt.legend()
#required n_neighbors = 2

1
cols_mask_dim = (75, 1), rows_mask_dim = (75, 1)
2
cols_mask_dim = (75, 2), rows_mask_dim = (75, 2)
3
cols_mask_dim = (75, 3), rows_mask_dim = (75, 3)
4
cols_mask_dim = (75, 4), rows_mask_dim = (75, 4)
5
cols_mask_dim = (75, 5), rows_mask_dim = (75, 5)
6
cols_mask_dim = (75, 6), rows_mask_dim = (75, 6)
7
cols_mask_dim = (75, 7), rows_mask_dim = (75, 7)
8
cols_mask_dim = (75, 8), rows_mask_dim = (75, 8)
9
cols_mask_dim = (75, 9), rows_mask_dim = (75, 9)
10
cols_mask_dim = (75, 10), rows_mask_dim = (75, 10)
11
cols_mask_dim = (75, 11), rows_mask_dim = (75, 11)
12
cols_mask_dim = (75, 12), rows_mask_dim = (75, 12)
13
cols_mask_dim = (75, 13), rows_mask_dim = (75, 13)
14
cols_mask_dim = (75, 14), rows_mask_dim = (75, 14)
15
cols_mask_dim = (75, 15), rows_mask_dim = (75, 15)
16
cols_mask_dim = (75, 16), rows_mask_dim = (75, 16)
17
cols_mask_dim = (75, 17), rows_mask_dim = (75, 17)
18
cols_mask_dim = (75, 18), rows_mask_dim = (75, 18)
19
cols_mask_dim = (75, 19), rows_mask_dim = (75, 19)
2

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (75,76) (75,75) 

In [108]:
neighbors = range(1, 100)
accur = []
for i in neighbors:
    my_clf = KNeighborsClassifier(n_neighbors=i)
    my_clf.fit(X_train, y_train)
    accur.append(my_clf.score(X_test, y_test))

plt.plot(neighbors, accur, color="blue", label="score")
plt.legend()

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 75, n_neighbors = 76

**MNIST**

In [3]:
mnist = fetch_mldata('MNIST original')

In [4]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.0002, stratify=mnist.target)

In [5]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [6]:
%time clf.fit(X_train, y_train)

CPU times: user 45.1 s, sys: 185 ms, total: 45.3 s
Wall time: 45.3 s


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [7]:
%time my_clf.fit(X_train, y_train)

CPU times: user 38.2 ms, sys: 28 ms, total: 66.2 ms
Wall time: 65.3 ms


In [8]:
%time clf.predict(X_test)

CPU times: user 2.24 s, sys: 3.95 ms, total: 2.25 s
Wall time: 2.25 s


array([3., 0., 2., 4., 1., 1., 7., 9., 8., 7., 6., 2., 3., 5.])

In [9]:
%time my_clf.predict(X_test)

cols_mask_dim = (14, 2), rows_mask_dim = (14, 2)
CPU times: user 2.2 s, sys: 2.29 s, total: 4.49 s
Wall time: 6.38 s


array([7., 3., 7., 1., 1., 1., 7., 7., 2., 4., 1., 7., 7., 5.])

In [10]:
%time clf.predict_proba(X_test)

CPU times: user 2.31 s, sys: 125 ms, total: 2.44 s
Wall time: 3.08 s


array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [11]:
%time my_clf.predict_proba(X_test)

CPU times: user 2.11 s, sys: 1.47 s, total: 3.57 s
Wall time: 3.75 s


array([[0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0.5],
       [0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0.5, 0. , 0. ],
       [0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. ],
       [0. , 0.5, 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. ]])

In [12]:
clf.score(X_test, y_test)

1.0

In [13]:
my_clf.score(X_test, y_test)

cols_mask_dim = (14, 2), rows_mask_dim = (14, 2)


0.2857142857142857

Задание 5

In [1]:
# seaborn визуализация статистических данных
# matplotlib построение графиков
# train_test_split разбиение выборки на тестовую и обучающую
# Pipelin%load_ext e (from sklearn.pipeline import Pipeline) ? (выполнение операций в определенной последовательности)
# StandardScaler (from sklearn.preprocessing import StandardScaler) приводит выборку к нулевому матожиданию и единичной дисперсии
# ClassifierMixin базовый класс классификаторов в sklearn
# NeighborsBase базовый класс метода ближайших соседей
# KNeighborsMixin загружает обучающую выборку для метода k-ближайших соседей
# SupervisedIntegerMixin загружает обучающую выборку 